# 5.1 선형 svm 분류

caution!! SVM은 특성의 스케일에 민감 (support vector의 특성상 large margin을 추구하므로)-> 사이킷런의 StandardScaler 사용

## 5.1 소프트 마진분류

모든 샘플이 올바르게 분류 -> 하드 마진 분류(hard margin classification) : 모든 데이터를 선형적으로 분류해야 제대로 작동하고 이상치에 민감
    
마진 오류를 적절히 설정함 -> 소프트 마진 분류(soft margin classification) 

In [7]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris=datasets.load_iris()
X=iris["data"][:,(2,3)]
y=(iris["target"]==2).astype(np.float64)

svm_clf=Pipeline([
    ("scalar",StandardScaler()),
    ("linear_svc",LinearSVC(C=1,loss="hinge")),
])

svm_clf.fit(X,y)


Pipeline(memory=None,
         steps=[('scalar',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linear_svc',
                 LinearSVC(C=1, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)

In [8]:
svm_clf.predict([[5.5, 1.7]])

array([1.])

## 5.2 비선형 SVM 분류

선형적으로 분류가 안되는 경우가 많으므로 비선형적인 피쳐를 추가해줘서 분류를 쉽게함

In [15]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

X,y = make_moons(n_samples=100, noise=0.15)
polynomial_svm_clf=Pipeline([
    ("poly_features",PolynomialFeatures(degree=3)),
    ("scaler",StandardScaler()),
    ("svm_clf",LinearSVC(C=10,loss="hinge"))
]) 
# PolynomialFeatures-> n차항의 피쳐로 추가해 선형분류를 쉽게 만들어줌( 모든 머신러닝 알고리즘에서 잘통하나 피쳐가 늘어나므로 모델을 느리게 만든다.)
polynomial_svm_clf.fit(X,y)


C:\Anaconda\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('poly_features',
                 PolynomialFeatures(degree=3, include_bias=True,
                                    interaction_only=False, order='C')),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 LinearSVC(C=10, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)

## 5.2.1 다항식 커널

In [20]:
from sklearn.svm import SVC
poly_kernel_svm_clf =Pipeline([
    ("scalar",StandardScaler()),
    ("svm_clf",SVC(kernel="poly",degree=3,coef0=1,C=5)) # C-> 마진오류 
])
poly_kernel_svm_clf.fit(X,y)# degree=3 이므로 3차 다항식 커널을 사용해서 분류기를 훈련
# coef0 는 모델이 높은 차수와 낮은 차수에 얼마나 영향을 받을 지 조정

Pipeline(memory=None,
         steps=[('scalar',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 SVC(C=5, break_ties=False, cache_size=200, class_weight=None,
                     coef0=1, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='poly', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

# 5.2.2 유사도 특성

비선형적인 특성을 다루기 위해서 각 샘플이 특정 랜드마크(landmark)와 얼마나 닮았는지 측정하는 유사도 함수(similarity function)로 계산한 특성 추가

유사도 함수는 가우시안(=radial basis function)로 보통 설정 , landmark는 데이터셋의 모든 샘플위에 랜드마크 설정

-> 장점 :차원이 커지고 훈련셋이 선형으로 구분됨 
    
    단점: n개의 특성 m개의 데이터 -> m개의 특성, m개의 데이터

In [29]:
 ## 5.2.3 가우시안 RBF 커널
rbf_kernel_svm_clf=Pipeline([
    ("scaler",StandardScaler()),
    ("svm_clf",SVC(kernel="rbf", gamma=5, C=0.001))
])
rbf_kernel_svm_clf.fit(X,y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 SVC(C=0.001, break_ties=False, cache_size=200,
                     class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma=5,
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

gamma: 증가할 시 -> 각 샘플의 영향 범위가 작아짐(심해지면 오버피팅)
       
       감소할 시 -> 더 일반적이게 됨 (심해지면 언더피팅)
       
선형커널->가우시안 커널-> 다른 커널

일반적으로 다른 커널은 잘 사용되지않음

문자열 커널(문자열 서브시퀀스 커널, 레벤슈타인 커널)이 텍스트 문서나 DNA 염기서열을 분류할떄 사용됨 

(두 문자열의 유사도를 비교하는 함수) 

# 5.3 svm 회귀

In [31]:
from sklearn.svm import LinearSVR

svm_reg =LinearSVR(epsilon=1.5)
svm_reg.fit(X,y)

LinearSVR(C=1.0, dual=True, epsilon=1.5, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=None, tol=0.0001, verbose=0)

In [32]:
from sklearn.svm import SVR

svm_poly_reg=SVR(kernel="poly", degree=2, C=100, epsilon=0.1)

svm_poly_reg.fit(X,y) # 다항으로 커널트릭 사용

SVR(C=100, cache_size=200, coef0=0.0, degree=2, epsilon=0.1, gamma='scale',
    kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)